In [2]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 22.15 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [3]:
%cd shap-e

/content/shap-e


In [81]:
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 16.5 MB/s eta 0:00:00


In [4]:
pip install -e .

Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-_baa0oaa/clip_400a1c0d2b5c4a80ae669c16032145d1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-_baa0oaa/clip_400a1c0d2b5c4a80ae669c16032145d1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━

In [5]:
!pip install rembg
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00


In [6]:
!pip install onnxruntime

In [19]:
from shap_e.util.image_util import load_image
from google. colab.patches import cv2_imshow
from google.colab import drive
from rembg import remove
import torch


In [42]:
import torch
from rembg import remove
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image
from shap_e.util.notebooks import decode_latent_mesh
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)

diffusion = diffusion_from_config(load_config('diffusion'))

In [77]:

def image_to3d(image_path,output_path):
  from google. colab.patches import cv2_imshow
  from google.colab import drive
  from rembg import remove
  model = load_model('image300M', device=device)
  drive.mount('/content/drive')

  # To get the best result, you should remove the background and show only the object of interest to the model.

  img = load_image(image_path)
  # Output path
  output_path = output_path+('output_image.png')
  # Remove background
  output_image = remove(img)
  # Save the result
  output_image.save(output_path)

  batch_size = 1
  guidance_scale = 3.0
  from google. colab.patches import cv2_imshow
  from google.colab import drive
  # Mount Google Drive to access your files
  drive.mount('/content/drive')

  # To get the best result, you should remove the background and show only the object of interest to the model.
  image = load_image(output_path)

  latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,)
  render_mode = 'stf' # you can change this to 'stf' for mesh rendering
  size = 64 # this is the size of the renders; higher values take longer to render.
  cameras = create_pan_cameras(size, device)
  for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))
  for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_meshio_{i}.obj', 'w') as f:
        t.write_obj(f)

In [64]:
image_to3d()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAANTT0s7Ny87JxcfHxtG+p8PAvb6+vrq4t+C0dt+yc92ydN2vcd…

1
111


In [72]:
def text_to_3d(prompt):
  batch_size = 1
  guidance_scale = 15.0

  model = load_model('text300M', device=device)
  latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)
  render_mode = 'nerf' # you can change this to 'stf'
  size = 64 # this is the size of the renders; higher values take longer to render.

  cameras = create_pan_cameras(size, device)
  for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))
    from shap_e.util.notebooks import decode_latent_mesh

  for i, latent in enumerate(latents):
      t = decode_latent_mesh(xm, latent).tri_mesh()
      with open(f'example_mesh_{i}.ply', 'wb') as f:
          t.write_ply(f)
      with open(f'example_mesh_{i}.obj', 'w') as f:
          t.write_obj(f)

In [67]:
text_to_3d()

  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAHprDnhqDW1cDmpcDFROO1pKCllKCVpJEFZGElNFDlFCD09AC0…

In [90]:
import trimesh

mesh = trimesh.load('/content/shap-e/example_mesh_0.obj')
mesh.show()


In [88]:
if __name__ == "__main__":
    choice = input("Please enter 'text' to proceed with text-to-3D conversion, or enter any other input to proceed with image-to-3D conversion: ")

    if choice.lower() == "text":
        prompt = input("Please enter the text prompt: ")
        text_to_3d(prompt)
    else:
        image_path = input("Please enter the image file path: ")
        print("Kindly specify the directory where the 3D output should be saved.")
        output_path = input("Please enter the destination path for saving the output: ")
        image_to_3d(image_path, output_path)
    visual=input("specify the current directeryfor visualization")
    mesh = trimesh.load(visual)
    mesh.show()
    mesh = trimesh.load(visual)
    mesh.show()

Please enter 'text' to proceed with text-to-3D conversion, or enter any other input to proceed with image-to-3D conversion: text
Please enter the text prompt: a plane with legs


  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIYAAO7v7+7u7u3t7u3s7Ozs7ezs7Ozr7Ozr6+vr7Ovr6+vq6+vq6u…

specify the current directeryfor visualization/content/shap-e/example_mesh_0.obj
